# Align images using astroalign
Notebook which aligns two images taken with the PESTO instrument AFTER running through the reduction pipeline [here](https://reductionpipeline.readthedocs.io/en/latest/).

In [21]:
from astropy.io import fits
import matplotlib.pyplot as plt
import astroalign as aa
import numpy as np
from astropy.wcs import WCS

In [19]:
NGC_g = fits.open('NGC5866_g.fits')[0].data.byteswap().newbyteorder()
NGC_ha = fits.open('NGC5866_ha.fits')[0].data.byteswap().newbyteorder()
NGC_i = fits.open('NGC5866_i.fits')[0].data.byteswap().newbyteorder()

## Align
We will now realign the g and i bands to be concident with ha

In [20]:
NGC_g_aligned, footprint = aa.register(NGC_g, NGC_ha)
NGC_i_aligned, footprint = aa.register(NGC_i, NGC_ha)

## Save
Now we save the realigned images as fits images using the ha WCS

In [22]:
hdr = WCS(fits.open('NGC5866_ha.fits')[0].header).to_header()
fits.writeto('NGC5866_g_aligned.fits', NGC_g_aligned, hdr)
fits.writeto('NGC5866_i_aligned.fits', NGC_i_aligned, hdr)